In [1]:
import numpy as np

def load_motion_from_dataset(name):
    data = np.load(r'HumanML3D\new_joints\\' + name + '.npy', allow_pickle=True)
    text = np.loadtxt(r'HumanML3D\texts\\' + name + '.txt', dtype='str', comments="#", delimiter=",", usecols=0)[0]
    print(text)
    #print(data.shape)

    motion = data

    return text, motion

In [2]:
import os

dir_path = r"HumanML3D\new_joints\\"

# list to store files
res = []

i = 0
# Iterate directory
for path in os.listdir(dir_path):
    # check if current path is a file
    if os.path.isfile(os.path.join(dir_path, path)):
        res.append(path[:-4])
        text, motion = load_motion_from_dataset(path[:-4])
        print(motion)
    if i == 0:
        break

a man kicks something or someone with his left leg.
[[[ 0.0000000e+00  9.4151199e-01  0.0000000e+00]
  [ 6.2321343e-02  8.6119002e-01 -1.6990200e-02]
  [-5.5334441e-02  8.4660774e-01 -2.3730025e-03]
  ...
  [-2.1101892e-01  1.0666605e+00 -3.9468467e-02]
  [ 2.6309258e-01  8.2468820e-01  4.4097662e-02]
  [-2.7899861e-01  8.3389765e-01  7.8994945e-02]]

 [[ 8.2115037e-04  9.4221103e-01 -2.4421725e-03]
  [ 6.3244611e-02  8.6205578e-01 -1.9839937e-02]
  [-5.4350927e-02  8.4721702e-01 -4.9981074e-03]
  ...
  [-2.1169032e-01  1.0679228e+00 -3.4716845e-02]
  [ 2.6594034e-01  8.2585931e-01  4.7512941e-02]
  [-2.7718514e-01  8.3534062e-01  8.5488237e-02]]

 [[ 2.6924799e-03  9.4400620e-01 -7.7703120e-03]
  [ 6.5317206e-02  8.6424065e-01 -2.6204795e-02]
  [-5.2142885e-02  8.4883380e-01 -1.0876785e-02]
  ...
  [-2.1393704e-01  1.0710016e+00 -2.6905689e-02]
  [ 2.7320394e-01  8.3016169e-01  5.7480182e-02]
  [-2.7432778e-01  8.3984303e-01  9.8601252e-02]]

 ...

 [[ 2.2440001e-02  9.4192517e-01  2.

In [3]:
import numpy as np

# Define a kinematic tree for the skeletal struture
kit_kinematic_chain = [[0, 11, 12, 13, 14, 15], [0, 16, 17, 18, 19, 20], [0, 1, 2, 3, 4], [3, 5, 6, 7], [3, 8, 9, 10]]

kit_raw_offsets = np.array(
    [
        [0, 0, 0],
        [0, 1, 0],
        [0, 1, 0],
        [0, 1, 0],
        [0, 1, 0],
        [1, 0, 0],
        [0, -1, 0],
        [0, -1, 0],
        [-1, 0, 0],
        [0, -1, 0],
        [0, -1, 0],
        [1, 0, 0],
        [0, -1, 0],
        [0, -1, 0],
        [0, 0, 1],
        [0, 0, 1],
        [-1, 0, 0],
        [0, -1, 0],
        [0, -1, 0],
        [0, 0, 1],
        [0, 0, 1]
    ]
)

t2m_raw_offsets = np.array([[0,0,0],
                           [1,0,0],
                           [-1,0,0],
                           [0,1,0],
                           [0,-1,0],
                           [0,-1,0],
                           [0,1,0],
                           [0,-1,0],
                           [0,-1,0],
                           [0,1,0],
                           [0,0,1],
                           [0,0,1],
                           [0,1,0],
                           [1,0,0],
                           [-1,0,0],
                           [0,0,1],
                           [0,-1,0],
                           [0,-1,0],
                           [0,-1,0],
                           [0,-1,0],
                           [0,-1,0],
                           [0,-1,0]])

t2m_kinematic_chain = [[0, 2, 5, 8, 11], [0, 1, 4, 7, 10], [0, 3, 6, 9, 12, 15], [9, 14, 17, 19, 21], [9, 13, 16, 18, 20]]
t2m_left_hand_chain = [[20, 22, 23, 24], [20, 34, 35, 36], [20, 25, 26, 27], [20, 31, 32, 33], [20, 28, 29, 30]]
t2m_right_hand_chain = [[21, 43, 44, 45], [21, 46, 47, 48], [21, 40, 41, 42], [21, 37, 38, 39], [21, 49, 50, 51]]


kit_tgt_skel_id = '03950'

t2m_tgt_skel_id = '000021'

In [4]:
def plot_3d_motion(joints, dataset):

    #         return ax

    # (seq_len, joints_num, 3)
    #print(joints.shape)
    #print(len(joints))
    data = joints.copy().reshape(len(joints), -1, 3)

    # preparation related to specific datasets
    if dataset == 'kit':
        data *= 0.003  # scale for visualization
    elif dataset == 'humanml':
        data *= 1.3  # scale for visualization
    elif dataset in ['humanact12', 'uestc']:
        data *= -1.5 # reverse axes, scale for visualization

    #print(data[0].shape)

    MINS = data.min(axis=0).min(axis=0)
    MAXS = data.max(axis=0).max(axis=0)
    #print("MINS MAXS", [MINS, MAXS])

    frame_number = data.shape[0]
    #print("frame_number", frame_number)

    height_offset = MINS[1]
    data[:, :, 1] -= height_offset
    trajec = data[:, 0, [0, 2]] #x and y of joint 0
    """
    print("data max 0", data[-1, :, [0, 2]][0].max())
    print("data max 0", data[-1, :, [0, 2]][0].min())
    print("data max 1", data[-1, :, [0, 2]][1].max())
    print("data max 1", data[-1, :, [0, 2]][1].min())
    """
    maxx = data[-1, :, [0, 2]][0].max()
    minx = data[-1, :, [0, 2]][0].min()
    maxz = data[-1, :, [0, 2]][1].max()
    minz = data[-1, :, [0, 2]][1].min()

    np.save(save_path + "_ending_area.npy", np.array([maxx, minx, maxz, minz]))
    #print(trajec.shape)
    #print(trajec)
    #ax.scatter(trajec[:,0], trajec[:,1], np.array(120*[0]))
    

    data[..., 0] -= data[:, 0:1, 0]
    data[..., 2] -= data[:, 0:1, 2]

In [5]:
import os

fps=20
dataset = "humanml"
skeleton = kit_kinematic_chain if dataset == 'kit' else t2m_kinematic_chain

def render_motion(motion, skeleton):
    length = motion.shape[0]
    motion = motion.transpose(0, 1, 2)[:length]
    #print("motion shape render", motion.shape)

    plot_3d_motion(skeleton, motion)